In [40]:
import sys
import uproot

import os
import numpy as np
import pandas as pd

import argparse

In [41]:
part =  "gamma" #"muon" #"electron" #"pion_c"
path = "/data/user/adipilat/ParticleID/genEvts/"
dir_ = "ana"
tree = "hgc"
max_perlayer = 10
number_layers = 50

In [42]:
variableName = [
            'event','run','lumi',
            'cluster2d_layer',
            'cluster2d_energy',
            'cluster2d_eta',
            'cluster2d_phi',
            'cluster2d_pt',
            'cluster2d_x',
            'cluster2d_y',
            'cluster2d_z',
            'cluster2d_nhitCore',
            'cluster2d_nhitAll',
            'gen_energy',
            'gen_pdgid',
            'gen_daughters',
            'gen_phi',
            'gen_eta',
            'trackster_clusters'
            ]
newVars =["run","lumi","event","trackster","x","y","z","r","layer","E","nCore","nHits","id","genDR","gen_phi","gen_eta","phi","eta","pid","genE"]

In [43]:
print("Starting data production")
name = "4_" + part
file = path + part + "/" + name + ".root"

Starting data production


In [44]:
df = uproot.open(file)[dir_][tree].pandas.df(variableName,flatten=False)


num_events = np.unique(df["event"].values).shape[0]
xs = df["cluster2d_x"].values
ys = df["cluster2d_y"].values
zs = df["cluster2d_z"].values
es = df["cluster2d_energy"].values
ps = df["cluster2d_pt"].values
nh = df["cluster2d_nhitAll"].values
nc = df["cluster2d_nhitCore"].values
ll = df["cluster2d_layer"].values
ee = df["event"].values
lu = df["lumi"].values
ru = df["run"].values
    
sizes = [x.shape[0] for x in xs]
indices = [np.full((a[0]),a[1]) for a in zip(sizes,range(len(sizes)))]
        #gendau = [np.full((a[0]),a[1]) for a in zip(sizes,df["gen_daughters"].values)]

cphi = df["cluster2d_phi"].values
ceta = df["cluster2d_eta"].values
gpid = df["gen_pdgid"].values
gen = df["gen_energy"].values

gphi = [np.full((a[0]),a[1]) for a in zip(sizes,df["gen_phi"].values)]
geta = [np.full((a[0]),a[1]) for a in zip(sizes,df["gen_eta"].values)]
gpid = [np.full((a[0]),a[1]) for a in zip(sizes,df["gen_pdgid"].values)]
gen = [np.full((a[0]),a[1]) for a in zip(sizes,df["gen_energy"].values)]


tr = df["trackster_clusters"].values

In [45]:
idtrlist = []
# LayerClusters that don't belong to any Trackster will have TracksterId = 0. Real Tracksters have the TracksterId > 0
for i in range(len(sizes)):
    idtrlist.append(np.array([0]*sizes[i]))
for i in range(len(tr)):
    for j in range(len(tr[i])):
        for item in tr[i][j]:
            idtrlist[i][item] = j + 1

idtr = np.array(idtrlist)

In [46]:
rs = [np.sqrt(f[0]**2+f[1]**2) for f in zip(xs,ys)]

drs = [np.sqrt((a[0]-a[1])**2 + (a[2]-a[3])**2) for a in zip(gphi,cphi,geta,ceta)]


In [59]:
XS = np.array([item for sublist in xs for item in sublist])
YS = np.array([item for sublist in ys for item in sublist])
ZS = np.array([item for sublist in zs for item in sublist])
RS = np.array([item for sublist in rs for item in sublist])
LL = np.array([item for sublist in ll for item in sublist])
ES = np.array([item for sublist in es for item in sublist])
NC = np.array([item for sublist in nc for item in sublist])
NH = np.array([item for sublist in nh for item in sublist])
II = np.array([item for sublist in indices for item in sublist])
DRS = np.array([item for sublist in drs for item in sublist])
GPHI = np.array([item for sublist in gphi for item in sublist])
GETA = np.array([item for sublist in geta for item in sublist])
GPID = np.array([item for sublist in gpid for item in sublist])
GEN = np.array([item for sublist in gen for item in sublist])
CPHI = np.array([item for sublist in cphi for item in sublist])
CETA = np.array([item for sublist in ceta for item in sublist])


SS = [np.full((s,),s) for s in sizes]
EE = [np.full((s,),i) for i,s in zip(ee,sizes)]
LU = [np.full((s,),i) for i,s in zip(lu,sizes)]
RU = [np.full((s,),i) for i,s in zip(ru,sizes)]


SS = np.array([item for sublist in SS for item in sublist])
EE = np.array([item for sublist in EE for item in sublist])
LU = np.array([item for sublist in LU for item in sublist])
RU = np.array([item for sublist in RU for item in sublist])

TR = np.array([item for sublist in idtr for item in sublist])

datas = np.vstack((RU,LU,EE,TR,XS,YS,ZS,RS,LL,ES,NC,NH,II,DRS,GPHI,GETA,CPHI,CETA,GPID,GEN)).T

346506

In [58]:
df = pd.DataFrame(datas,columns=newVars)
df = df.sort_values(["event","trackster","layer","E"],ascending=[True,True,True,False]).reset_index(drop=True)

,run,lumi,event,trackster,x,y,z,r,layer,E,nCore,nHits,id,genDR,gen_phi,gen_eta,phi,eta,pid,genE
0,1.0,1.0,7.0,0.0,-6.800393,-141.415436,422.199005,141.578857,39.0,0.650759,3.0,3.0,0.0,0.007459,-1.616928,1.805555,-1.618847,1.812763,13.0,197.162338
1,1.0,1.0,7.0,0.0,-4.584855,-108.758011,325.072754,108.854607,3.0,0.015671,1.0,1.0,0.0,0.009440,-1.616928,1.805555,-1.612928,1.814106,13.0,197.162338
2,1.0,1.0,7.0,0.0,-4.584855,-108.758011,326.017303,108.854607,4.0,0.155909,1.0,1.0,0.0,0.011989,-1.616928,1.805555,-1.612928,1.816858,13.0,197.162338
3,1.0,1.0,7.0,0.0,-5.356941,-109.749290,328.042755,109.879951,5.0,0.184036,3.0,3.0,0.0,0.008695,-1.616928,1.805555,-1.619568,1.813840,13.0,197.162338
4,1.0,1.0,7.0,0.0,-6.062663,-111.587738,328.987274,111.752312,6.0,0.018484,2.0,2.0,0.0,0.009559,-1.616928,1.805555,-1.625074,1.800554,13.0,197.162338


In [55]:
df.to_hdf(path + part + ".h5","data",complevel=0)